In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [2]:
!wget -c https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py
!wget https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
! unzip -q flower_data.zip && rm flower_data.zip

--2019-01-04 13:47:53--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2019-01-04 13:47:55--  https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.137.70
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.137.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 311442766 (297M) [application/zip]
Saving to: ‘flower_data.zip.1’

flower_data.zip.1   100%[===================>] 297.01M  36.5MB/s    in 8.8s    

2019-01-04 13:48:05 (33.9 MB/s) - ‘flower_data.zip.1’ saved [311442766/31144

In [3]:
# we need pillow version of 5.3.0
# we will uninstall the older version first
!pip uninstall -y Pillow
# install the new one
!pip install Pillow==5.3.0
# import the new one
import PIL
print(PIL.PILLOW_VERSION)
# this should print 5.3.0. If it doesn't, then restart your runtime:
# Menu > Runtime > Restart Runtime

Uninstalling Pillow-5.3.0:
  Successfully uninstalled Pillow-5.3.0
  Using cached https://files.pythonhosted.org/packages/62/94/5430ebaa83f91cc7a9f687ff5238e26164a779cca2ef9903232268b0a318/Pillow-5.3.0-cp36-cp36m-manylinux1_x86_64.whl
5.3.0


In [17]:
# Imports here
import time
import json
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
from collections import OrderedDict
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [0]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

dirs = {'train': train_dir, 
        'valid': valid_dir}

In [0]:
# TODO: Define your transforms for the training and validation sets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(45),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
}

# TODO: Load the datasets with ImageFolder
image_datasets = {x: datasets.ImageFolder(dirs[x], transform=data_transforms[x]) for x in ['train', 'valid']}

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8, shuffle=True) for x in ['train', 'valid']}
dataset_sizes = {x: len(image_datasets[x]) 
                              for x in ['train', 'valid']}
class_names = image_datasets['train'].classes

In [8]:
!wget -c https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json

--2019-01-04 13:48:43--  https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
Resolving github.com (github.com)... 140.82.118.3, 140.82.118.4
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/udacity/pytorch_challenge/master/cat_to_name.json [following]
--2019-01-04 13:48:43--  https://raw.githubusercontent.com/udacity/pytorch_challenge/master/cat_to_name.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2218 (2.2K) [text/plain]
Saving to: ‘cat_to_name.json’

cat_to_name.json    100%[===================>]   2.17K  --.-KB/s    in 0s      

2019-01-04 13:48:43 (30.8 MB/s) - ‘cat_to_name.json’ saved [2218/2218]



In [0]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [10]:
model = models.resnet152(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.torch/models/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [00:03<00:00, 79474159.81it/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2048, 1024)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(1024, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

for param in model.parameters():
    param.requires_grad = False

In [0]:
model.fc = classifier
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters())
model = model.cuda()

In [0]:
def train_model(model, criteria, optimizer, scheduler, num_epochs=25, device='cuda'):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
# Criteria NLLLoss which is recommended with Softmax final layer
criteria = nn.NLLLoss()
# Observe that all parameters are being optimized
optim = optim.Adam(model.fc.parameters(), lr=0.001)
# Decay LR by a factor of 0.1 every 4 epochs
sched = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
# Number of epochs
eps=10

In [16]:
model_ft = train_model(model, criteria, optim, sched, eps, 'cuda')

Epoch 0/9
----------
train Loss: 2.8143 Acc: 0.3524
valid Loss: 1.2318 Acc: 0.6589

Epoch 1/9
----------
train Loss: 1.2481 Acc: 0.6696
valid Loss: 0.5880 Acc: 0.8619

Epoch 2/9
----------
train Loss: 0.9590 Acc: 0.7360
valid Loss: 0.4611 Acc: 0.8839

Epoch 3/9
----------
train Loss: 0.8029 Acc: 0.7764
valid Loss: 0.4507 Acc: 0.8851

Epoch 4/9
----------
train Loss: 0.7382 Acc: 0.7958
valid Loss: 0.3807 Acc: 0.8985

Epoch 5/9
----------
train Loss: 0.6926 Acc: 0.8033
valid Loss: 0.3688 Acc: 0.9010

Epoch 6/9
----------
train Loss: 0.6425 Acc: 0.8211
valid Loss: 0.3335 Acc: 0.8985

Epoch 7/9
----------
train Loss: 0.6191 Acc: 0.8223
valid Loss: 0.2854 Acc: 0.9205

Epoch 8/9
----------
train Loss: 0.6194 Acc: 0.8272
valid Loss: 0.2807 Acc: 0.9267

Epoch 9/9
----------
train Loss: 0.6284 Acc: 0.8230
valid Loss: 0.3291 Acc: 0.9144

Training complete in 45m 54s
Best val Acc: 0.926650


In [0]:
for param in model.fc.parameters():
    param.requires_grad = True

In [0]:
optim = optim.Adagrad(model.fc.parameters(), lr=0.00001)

In [21]:
model_ft = train_model(model, criteria, optim, sched, eps, 'cuda')

Epoch 0/9
----------
train Loss: 0.6731 Acc: 0.8082
valid Loss: 0.2977 Acc: 0.9303

Epoch 1/9
----------
train Loss: 0.6839 Acc: 0.8060
valid Loss: 0.2947 Acc: 0.9242

Epoch 2/9
----------
train Loss: 0.6825 Acc: 0.8147
valid Loss: 0.2793 Acc: 0.9254

Epoch 3/9
----------
train Loss: 0.6514 Acc: 0.8135
valid Loss: 0.2786 Acc: 0.9328

Epoch 4/9
----------
train Loss: 0.6673 Acc: 0.8107
valid Loss: 0.3011 Acc: 0.9218

Epoch 5/9
----------
train Loss: 0.6603 Acc: 0.8159
valid Loss: 0.2904 Acc: 0.9315

Epoch 6/9
----------
train Loss: 0.6553 Acc: 0.8161
valid Loss: 0.2935 Acc: 0.9242

Epoch 7/9
----------
train Loss: 0.6589 Acc: 0.8111
valid Loss: 0.2754 Acc: 0.9242

Epoch 8/9
----------
train Loss: 0.6497 Acc: 0.8199
valid Loss: 0.2884 Acc: 0.9279

Epoch 9/9
----------
train Loss: 0.6549 Acc: 0.8152
valid Loss: 0.2867 Acc: 0.9279

Training complete in 50m 7s
Best val Acc: 0.932763


In [0]:
model.class_to_idx = image_datasets['train'].class_to_idx
model.cpu()
torch.save({'arch': 'resnet152',
            'state_dict': model.state_dict(), 
            'class_to_idx': model.class_to_idx}, 
            'checkpoint-resnet152.pth')

In [0]:
import os
statinfo = os.stat('checkpoint-resnet152.pth')
statinfo

os.stat_result(st_mode=33188, st_ino=9569578, st_dev=46, st_nlink=1, st_uid=0, st_gid=0, st_size=242133403, st_atime=1546524103, st_mtime=1546524103, st_ctime=1546524103)

In [0]:
statinfo.st_size

242133403

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
uploaded = drive.CreateFile({'title': 'checkpoint-resnet152.pth'})
uploaded.SetContentFile('checkpoint-resnet152.pth')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1HviuDlv514a_sv2Qfqjsg1hqLgFObW56
